# Building your PLUMED input file within Python

Here you can see some example showing how to use the Python input builder for PLUMED. The goal of this builder is to simplify the construction of PLUMED input files in a Python workflow by providing:
1. Autocompletion of action names and arguments.
2. Fast access to PLUMED documentation using Python docstrings.
3. Automatic translation of python lists, tuples, and ranges, also taking care of PLUMED braces.
4. Interactive detection of errors.

In [1]:
%autosave 0
%qtconsole

Autosave disabled


The first thing that you should do is to import the relevant Python module and construct an InputBuilder object

In [2]:
import plumed
ib=plumed.InputBuilder()

The `ib` object contains a full dictionary of available actions. Currently, this dictionary is built using the VIM syntax file, and the path to the VIM syntax file is found by looking at the command `plumed` in your execution path. Make sure that the `plumed` command can be executed. **The way the dictionary is constructed will change in the future**, but this seems a reasonable temporary solution to allow you loading the dictionary of an already installed PLUMED package.

### Using the InputBuilder

At this point you will be able to use the `ib` object to create PLUMED input lines. Any PLUMED action can be created using a method of object `ib`, and arguments for that method can be used to supply the necessary keywords. The first argument by default will be interpreted as the label of the action, and can be omitted. All the other arguments are keyword-only arguments.

If you use this sheet interactively, notice that you can use `<TAB>` to complete both actions and arguments names, and that you can use `SHIFT+<TAB>` to access to a short documentation of the given action. This documentation is currently the same one installed in the VIM syntax.

Arguments can be provided as plain strings as in this example:

In [3]:
ib.TORSION("phi",ATOMS="5,7,9,15")

'phi: TORSION ATOMS=5,7,9,15\n'

Notice that if you use a space-separated list the input builder will provide braces for you. 

In [4]:
ib.TORSION("phi",ATOMS="5 7 9 15")

'phi: TORSION ATOMS={5 7 9 15}\n'

This means that you can easily pass more complex arguments such as those required for switching functions

In [5]:
ib.COORDINATION(GROUPA="1-10",GROUPB="11-20",SWITCH="RATIONAL NN=6 R_0=1")

'COORDINATION GROUPA=1-10 GROUPB=11-20 SWITCH={RATIONAL NN=6 R_0=1}\n'

Any tab or newline is replaced with a single space. Thus you can put long lists on multiple lines if you prefer

In [6]:
ib.COORDINATION(GROUPA="""
  1 2 3 4 5 6 7 8 9 10
  """,GROUPB="""
  11 12 13 14 15 16 17 18 19 20
  """,SWITCH="""
  RATIONAL NN=6 R_0=1
  """)

'COORDINATION GROUPA={   1 2 3 4 5 6 7 8 9 10   } GROUPB={   11 12 13 14 15 16 17 18 19 20   } SWITCH={   RATIONAL NN=6 R_0=1   }\n'

Flags can be passed as booleans

In [7]:
ib.DISTANCE("d",ATOMS="11 21",NOPBC=True)

'd: DISTANCE ATOMS={11 21} NOPBC\n'

Notice that passing a false flag or not passing the flag at all lead to the same result

In [8]:
ib.DISTANCE("d",ATOMS="11 21",NOPBC=False), ib.DISTANCE("d",ATOMS="11 21")

('d: DISTANCE ATOMS={11 21}\n', 'd: DISTANCE ATOMS={11 21}\n')

### Numerical arguments
So far we passed arguments as strings. If you pass them as numbers, they will just be formatted into strings:

In [9]:
ib.METAD(ARG="phi psi",PACE=500,HEIGHT=1.2,SIGMA="0.35 0.35",FILE="HILLS")

'METAD ARG={phi psi} FILE=HILLS HEIGHT=1.2 PACE=500 SIGMA={0.35 0.35}\n'

Notice that in this example the label has been omitted. As usual, PLUMED will provide a default label.

### Passing lists or tuples
It is also possible to pass ranges, lists, tuples, or numpy arrays. Anything that is iterable will be converted to a space separated list.

In [10]:
ib.GROUP("g",ATOMS=range(1,101))

'g: GROUP ATOMS={1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100}\n'

Notice that you can obtain an equivalent result with `ib.GROUP("g",ATOMS="1-100")`.

You can have lists of strings, lists of numbers, or also heterogenous lists. All combinations should work as expected

In [11]:
ib.METAD(ARG=("phi","psi"),PACE=500,HEIGHT=1.2,SIGMA=(0.35,"pi/8"),FILE="HILLS")

'METAD ARG={phi psi} FILE=HILLS HEIGHT=1.2 PACE=500 SIGMA={0.35 pi/8}\n'

### Numbered arguments

Some plumed actions require "numbered" arguments. You have two ways to pass those. One possibility is to append directly append the number of the argument to the keyword name:

In [12]:
ib.MOVINGRESTRAINT(ARG="d1",KAPPA0=0,KAPPA1=10.0,AT0=20,AT1=20,STEP0=1,STEP1=10000)

'MOVINGRESTRAINT ARG=d1 AT0=20 AT1=20 KAPPA0=0 KAPPA1=10.0 STEP0=1 STEP1=10000\n'

Notice that in this manner you will not be able to use python structures to list the values.

The `numbered()` function makes it simpler to specify multiple values. You can pass multiple a list/tuple:

In [13]:
ib.MOVINGRESTRAINT(ARG="d1",KAPPA=ib.numbered([0,10.0]),AT=ib.numbered([20,20]),STEP=ib.numbered([1,10000]))

'MOVINGRESTRAINT ARG=d1 AT0=20 AT1=20 KAPPA0=0 KAPPA1=10.0 STEP0=1 STEP1=10000\n'

If you want to pass a single element you should pass it as a list with one element (see next example).

Notice that in some cases it is not necessary to insert all the indeces. E.g., for a `MOVINGRESTRAINT` you might want to skip some `AT` values if the restraint is not expected to move in the time window. For maximum flexibility you can pass a dictionary with integer keys

In [14]:
ib.MOVINGRESTRAINT(ARG="d1",KAPPA=ib.numbered([100]),AT=ib.numbered({0:0.0,2:10.0}),STEP=ib.numbered((0,10,20,30)))

'MOVINGRESTRAINT ARG=d1 AT0=0.0 AT2=10.0 KAPPA0=100 STEP0=0 STEP1=10 STEP2=20 STEP3=30\n'

The command will be correctly formatted also when you have lists/tuples of lists/tuples.

In [15]:
ib.MOVINGRESTRAINT(ARG="d1,d2",
                     KAPPA=ib.numbered([11]),
                     AT=ib.numbered(((0.0,1.0),(2.0,3.0))),
                     STEP=ib.numbered((0,100)))

'MOVINGRESTRAINT ARG=d1,d2 AT0={0.0 1.0} AT1={2.0 3.0} KAPPA0=11 STEP0=0 STEP1=100\n'

Strings can be mixed with numbers also in this case

In [16]:
ib.MOVINGRESTRAINT(ARG="d1,d2",
                     KAPPA=ib.numbered([100]),AT=ib.numbered(([0.0,"pi"],[2.0,"pi"])),
                     STEP=ib.numbered((0,100)))

'MOVINGRESTRAINT ARG=d1,d2 AT0={0.0 pi} AT1={2.0 pi} KAPPA0=100 STEP0=0 STEP1=100\n'

### Replica syntax

In case you want to use the replica syntax (introduced in plumed 2.4) you can use the `replicas` shortcut. You can pass multiple arguments or lists/tuples. Here the restraints will be located at 0, 1, 2, and 3 in four replicas:

In [17]:
ib.RESTRAINT(ARG="d1",KAPPA=10,AT=ib.replicas((0.0,1.0,2.0,3.0)))

'RESTRAINT ARG=d1 AT=@replicas:{0.0 1.0 2.0 3.0} KAPPA=10\n'

Here the restraints will be generated as equally spaced:

In [18]:
import numpy as np
ib.RESTRAINT(ARG="d1",KAPPA=10,AT=ib.replicas(np.linspace(3.0,5.0,17)))

'RESTRAINT ARG=d1 AT=@replicas:{3.0 3.125 3.25 3.375 3.5 3.625 3.75 3.875 4.0 4.125 4.25 4.375 4.5 4.625 4.75 4.875 5.0} KAPPA=10\n'

The command will be correctly formatted also when you have lists/tuples of lists/tuples. Here the restraints will be located at (0,1) in first replica and in (10,11) in second replica


In [19]:
ib.RESTRAINT(ARG="d1,d2",KAPPA=(10,10),AT=ib.replicas(([0.0,1.0],[10.0,11.0])))

'RESTRAINT ARG=d1,d2 AT=@replicas:{{0.0 1.0} {10.0 11.0}} KAPPA={10 10}\n'

## Atom selection

PLUMED has a number of shortcuts for atom selection (e.g. `@phi-1`). You might want to use the `ib.at` tool to build them. For instance:

In [20]:
ib.at.phi(6)

'@phi-6'

You can pass a chain as well, and both residue and chain can be lists/tuples (in which case they will be iterated). Chains can be strings or numbers. In case they are numbers they will be appended a `_` to be distinguished by the residue number.

In [21]:
ib.at.phi(range(1,11),"A")

'@phi-A1 @phi-A2 @phi-A3 @phi-A4 @phi-A5 @phi-A6 @phi-A7 @phi-A8 @phi-A9 @phi-A10'

In [22]:
ib.at.phi(range(1,11),["A","B"])

'@phi-A1 @phi-A2 @phi-A3 @phi-A4 @phi-A5 @phi-A6 @phi-A7 @phi-A8 @phi-A9 @phi-A10 @phi-B1 @phi-B2 @phi-B3 @phi-B4 @phi-B5 @phi-B6 @phi-B7 @phi-B8 @phi-B9 @phi-B10'

In [23]:
ib.at.phi(4,[1,2])

'@phi-1_4 @phi-2_4'

Notice that available shortcuts (e.g. `phi`, `psi`, etc) are encoded as functions so that you can see them typing `ib.at.<TAB>`. Additionally, you might select individual atoms by directly calling the `at` object as in the following example:

In [24]:
ib.at("OW",range(20,40))

'@OW-20 @OW-21 @OW-22 @OW-23 @OW-24 @OW-25 @OW-26 @OW-27 @OW-28 @OW-29 @OW-30 @OW-31 @OW-32 @OW-33 @OW-34 @OW-35 @OW-36 @OW-37 @OW-38 @OW-39'

Finally, there are a few groups that do not take the residue/chain argument. Those do not receive any argument

In [25]:
ib.at.mdatoms

'@mdatoms'

### Advanced atom selections with MDAnalysis

If [MDAnalysis](https://www.mdanalysis.org/) is installed, you can pass an atom Group to PLUMED. The indexes will be correctly translated to PLUMED serial numbers (1-base).

In [26]:
import MDAnalysis
universe=MDAnalysis.Universe("ref.pdb")
ib.GROUP("rna",ATOMS=universe.select_atoms("nucleic"))

/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/MDAnalysis/topology/PDBParser.py:330: UserWarning: Element information is absent or missing for a few atoms. Elements attributes will not be populated.
  warnings.warn("Element information is absent or missing for a few "


'rna: GROUP ATOMS={1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258}\n'

### Customizing the output
By default we use spaces as separator. In case you prefer commas you can use the option `comma_separator`

In [27]:
ib1=plumed.InputBuilder(comma_separator=True)
ib1.GROUP("g1",ATOMS=[1,2,3,4,5,6,7,8,9,10])

'g1: GROUP ATOMS=1,2,3,4,5,6,7,8,9,10\n'

## Passing arbitrary strings

You can also provide an arbitrary string using the `verbatim` argument. In this case you should take care of all the formatting issues (i.e., no braces will be added).

In [28]:
ib.RESTRAINT(ARG="d1,d2",verbatim="AT={10 20} KAPPA={5 6}")

'RESTRAINT ARG=d1,d2 AT={10 20} KAPPA={5 6}\n'

Entire arbitrary lines can be provided with the `verbatim` method. Might be useful for inserting comments or actions that are not correctly parsed in the InputBuilder dictionary

In [29]:
ib.verbatim("# here is a comment")

'# here is a comment\n'

In both these cases, newlines and tabs occuring within the passed string will be replaced with single spaces.

## Accessing the history

You can see the history of all the commands created with a builder by examining the `ib.history` list:

In [30]:
for line in ib.history:
    print(line)

phi: TORSION ATOMS=5,7,9,15
phi: TORSION ATOMS={5 7 9 15}
COORDINATION GROUPA=1-10 GROUPB=11-20 SWITCH={RATIONAL NN=6 R_0=1}
COORDINATION GROUPA={   1 2 3 4 5 6 7 8 9 10   } GROUPB={   11 12 13 14 15 16 17 18 19 20   } SWITCH={   RATIONAL NN=6 R_0=1   }
d: DISTANCE ATOMS={11 21} NOPBC
d: DISTANCE ATOMS={11 21}
d: DISTANCE ATOMS={11 21}
METAD ARG={phi psi} FILE=HILLS HEIGHT=1.2 PACE=500 SIGMA={0.35 0.35}
g: GROUP ATOMS={1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100}
METAD ARG={phi psi} FILE=HILLS HEIGHT=1.2 PACE=500 SIGMA={0.35 pi/8}
MOVINGRESTRAINT ARG=d1 AT0=20 AT1=20 KAPPA0=0 KAPPA1=10.0 STEP0=1 STEP1=10000
MOVINGRESTRAINT ARG=d1 AT0=20 AT1=20 KAPPA0=0 KAPPA1=10.0 STEP0=1 STEP1=10000
MOVINGRESTRAINT ARG=d1 AT0=0.0 AT2=10.0 KAPPA0=100 STEP0=0 

## Creating an input file

So far you have seen that the methods of `InputBuilder` return strings containing properly formatted plumed input lines. You might chain them in order to produce a long string of \n separated lines that you can then use in a real calculation. This is the suggested path if you use MD engines such as OpenMM-PLUMED.

In [31]:
s=""
s+=ib.verbatim("# compute CVs")
s+=ib.TORSION("phi",ATOMS="@phi-1")
s+=ib.TORSION("psi",ATOMS=[7,9,15,17])
s+=ib.verbatim("# do metadynamics")
s+=ib.METAD(ARG=("phi","psi"),PACE=500,HEIGHT=1.2,SIGMA=(0.35,0.35))
print(s)

# compute CVs
phi: TORSION ATOMS=@phi-1
psi: TORSION ATOMS={7 9 15 17}
# do metadynamics
METAD ARG={phi psi} HEIGHT=1.2 PACE=500 SIGMA={0.35 0.35}



In order to use the input file with an MD code reading a `plumed.dat` file (such as GROMACS) you can dump the input on a file with the following command

````
with open("plumed.dat","w") as file:
    print(s,file=file)
````
Alternatively, you can create an input builder using the `tofile` option.

In [32]:
with open("plumed.dat","w") as file:
    ibf=plumed.InputBuilder(tofile=file)
    ibf.verbatim("# compute CVs")
    ibf.TORSION("phi",ATOMS="@phi-1")
    ibf.TORSION("psi",ATOMS=[7,9,15,17])
    ibf.verbatim("# do metadynamics")
    ibf.METAD(ARG=("phi","psi"),PACE=500,HEIGHT=1.2,SIGMA=(0.35,0.35))

In [33]:
!cat plumed.dat

# compute CVs
phi: TORSION ATOMS=@phi-1
psi: TORSION ATOMS={7 9 15 17}
# do metadynamics
METAD ARG={phi psi} HEIGHT=1.2 PACE=500 SIGMA={0.35 0.35}


## Redirecting to a Plumed object

If you are using the python wrappers to PLUMED, you might want to redirect the plumed input to a Plumed object. Here is a minimal example. Notice that in order to properly parse the input plumed typically only needs to know the number of atoms. You might want to also set the Boltzmann constant or other quantities with the corresponding `cmd` instructions.

In [34]:
import plumed
plu=plumed.Plumed()
ibp=plumed.InputBuilder(toplumed=plu)
plu.cmd("setNatoms",10)
plu.cmd("init")
ibp.DISTANCE("d",ATOMS=(1,2))
ibp.RESTRAINT(ARG="d",AT=10,KAPPA=10)
ibp.verbatim("DISTANCE LABEL=cc ATOMS={10 2}")
del ibp
del plu
import gc
gc.collect()
# Notice that PLUMED output will go to the console!

516

## Building your input interactively

You can use both the `toplumed` and the `tofile` options simultaneously. This is interesting if you just want to build your input file interactively.

In [35]:
import plumed
plu=plumed.Plumed()
with open("plumed2.dat","w") as file:
    ibp=plumed.InputBuilder(toplumed=plu,tofile=file)
    plu.cmd("setNatoms",10)
    plu.cmd("init")
    ibp.DISTANCE("d1",ATOMS=(1,2))
    # This is an error (atom out of range:)
    # ibp.DISTANCE("d2",ATOMS=(1,11))
    ibp.DISTANCE("d2",ATOMS=(1,10))
del ibp
del plu
import gc
gc.collect()

516

The importat thing to notice is that if you make a mistake you can then continue to construct your input. At the end of the procedure, `plumed2.dat` will contain a plumed input file that contains only the correct lines and that can be then used in a simulation.

In [36]:
!cat plumed2.dat

d1: DISTANCE ATOMS={1 2}
d2: DISTANCE ATOMS={1 10}


## Analyzing a trajectory with mdtraj

To run the following example you need the mdtraj package.

We will first read the trajectory and initialize the input builder:

In [37]:
import mdtraj
import numpy as np
import re

traj = mdtraj.load("traj-whole.xtc",top="ref.pdb")

p=plumed.Plumed()
p.cmd("setNatoms",traj.n_atoms)
p.cmd("init")
builder=plumed.InputBuilder(toplumed=p)

Now you can construct your input file interactively. This is taken from the first Trieste tutorial. Atom selections are however done using mdtraj commands. Notice that **atom indexes need to be incremented by 1** (mdtraj numbers start from zero). Groups made with MDAnalysis (see above) do not suffer this problem since PLUMED can recognize them and treat them correctly.

The produced strings are immediately fed to the `p` object, so that an error is immediately detected.

In [38]:
builder.MOLINFO(STRUCTURE="ref.pdb")
# we do not use plumed groups here, but rather numpy arrays
rna=traj.top.select("resname =~ 'R.*'")+1
mg=traj.top.select("name MG")+1

assert(len(mg)==1)
owat = traj.top.select("water and name O") +1
hwat = traj.top.select("water and name H1 H2") +1
builder.GYRATION("r",ATOMS=rna)
builder.TORSION("c",ATOMS="@chi-1")
builder.COORDINATION("co",GROUPA=rna,GROUPB=owat,R_0=0.25)
builder.COORDINATION("ch",GROUPA=rna,GROUPB=hwat,R_0=0.25)
builder.CENTER("ce",ATOMS=rna)
builder.DISTANCE("d",ATOMS=(mg[0],"ce"))
builder.PRINT(FILE="COLVAR",ARG="r,c,co,ch,d")
# we then print the whole history so that you can see the commands:
for l in builder.history:
    print(l)

MOLINFO STRUCTURE=ref.pdb
r: GYRATION ATOMS={1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258}
c: TORSION ATOMS=@chi-1
co: CO

Finally, we process the trajectory. Notice that we always delete the plumed object at the end in order to finalize the calculation.

In [39]:
for i in range(0,traj.n_frames):
    p.cmd("setStep",i)
    coords=np.float64(traj.xyz[i,:,:])
    forces=np.zeros((traj.n_atoms,3),dtype=np.float64)
    virial=np.zeros((3,3),dtype=np.float64)
    traj.unitcell_vectors
    box=np.float64(traj.unitcell_vectors[i,:,:])
    masses=np.array([a.element.mass for a in traj.top.atoms],np.float64)
    p.cmd("setPositions",coords)
    p.cmd("setForces",forces)
    p.cmd("setVirial",virial)
    p.cmd("setBox",box)
    p.cmd("setMasses",np.array([a.element.mass for a in traj.top.atoms]))
    p.cmd("calc")
del p
del builder
gc.collect()

6133

And here you can see the result

In [40]:
!cat "COLVAR"

#! FIELDS time r c co ch d
#! SET min_c -pi
#! SET max_c pi
 0.000000 0.788694 -2.963153 206.199641 498.826542 0.595611
 0.000000 0.804101 -2.717304 206.427308 496.593226 0.951945
 0.000000 0.788769 -2.939333 206.744738 497.333158 1.014850
 0.000000 0.790232 -2.940730 209.695116 511.577794 1.249502
 0.000000 0.796395 3.050947 210.755477 504.675821 2.270682


### TODO
When using the `toplumed` option it would be possible to store in the InputBuilder the list of all the labels defined in the Plumed object. This would allow something like (**not implemented yet!**): 
````python
ib.DISTANCE("d",ATOMS=(1,2),COMPONENTS=True)
ib.RESTRAINT(ARG=ib.label["d.<TAB>
````
This could show `x` `y` and `z`. This requires modifying PLUMED to retrieve the defined label names.

Notice that something like this would be impossible when not using a `Plumed` object. Indeed, the only way to know which are the available components is to run a real PLUMED instance.

# Implementation notes

## Appending underscores to action names


There is a problem in jupyter autocompletion in old ipython implementations. Try the following definitions:

In [41]:
def A(one):
    pass
def AA(two):
    pass

Now type `A(<TAB>`. You might erroneously see both `one` and `two` as possible arguments. You can try to use `ib=plumed.InputBuilder(append_underscores=True)` to have modified action names (e.g. `ib.METAD__()`) to overcome issues of this type.

Notice that the flag `append_underscores` only affect the explicitly declared methods of the `InputBuilder` object. However, you are free to call the names with or without the underscores, as you prefer, irrespectively of the value of the `append_underscores` flag. In other words, this flag only controls what happens when you use autocompletion and is only useful if you have old jupyter notebooks.

## How are the functions created

A dictionary is first constructed by properly manipulating the VIM syntax file. The resulting dictionary is stored in `ib._vimdict`, so you can see it. The keys are all the available actions:

In [42]:
print(ib._vimdict.keys())

dict_keys(['ABMD', 'ADAPTIVE_PATH', 'ALPHABETA', 'ALPHARMSD', 'ANGLE', 'ANGLES', 'ANTIBETARMSD', 'AROUND', 'AVERAGE', 'BIASVALUE', 'BRIDGE', 'CALIBER', 'CAVITY', 'CELL', 'CENTER', 'CENTER_OF_MULTICOLVAR', 'COLLECT_FRAMES', 'COM', 'COMBINE', 'COMMITTOR', 'CONSTANT', 'CONTACTMAP', 'CONVERT_TO_FES', 'COORDINATION', 'COORDINATIONNUMBER', 'CS2BACKBONE', 'CUSTOM', 'DEBUG', 'DENSITY', 'DHENERGY', 'DIHCOR', 'DIMER', 'DIPOLE', 'DISTANCE', 'DISTANCES', 'DISTANCE_FROM_CONTOUR', 'DRMSD', 'DUMPATOMS', 'DUMPCUBE', 'DUMPDERIVATIVES', 'DUMPFORCES', 'DUMPGRID', 'DUMPMASSCHARGE', 'DUMPMULTICOLVAR', 'DUMPPROJECTIONS', 'EEFSOLV', 'EFFECTIVE_ENERGY_DRIFT', 'EMMI', 'ENDPLUMED', 'ENERGY', 'ENSEMBLE', 'ERMSD', 'EUCLIDEAN_DISSIMILARITIES', 'EXTENDED_LAGRANGIAN', 'EXTERNAL', 'EXTRACV', 'FAKE', 'FIND_CONTOUR', 'FIND_CONTOUR_SURFACE', 'FIND_SPHERICAL_CONTOUR', 'FIT_TO_TEMPLATE', 'FIXEDATOM', 'FLUSH', 'FOURIER_TRANSFORM', 'FRET', 'FUNCPATHGENERAL', 'FUNCPATHMSD', 'FUNCSUMHILLS', 'GHOST', 'GPROPERTYMAP', 'GRID_TO_X

The values provide further dictionaries that describe all the available keywords.

In [43]:
print(ib._vimdict["RESTRAINT"])

{'ARG': '(numbered)', 'AT': '(option)', 'KAPPA': '(option)', 'NUMERICAL_DERIVATIVES': '(flag)', 'SLOPE': '(option)', 'STRIDE': '(option)'}


Notice that there is not much information stored here. We might improve it in the future.

After this dictionary is read, it is used to dynamically create functions capable to generate an input file. These are standalone functions, and you can see their definitions in the dictionary `ib._functions`. Notice that this is python code that also defines the docstring of the function itself dynamically.

In [44]:
print(ib._functions["RESTRAINT"])

def RESTRAINT(self,LABEL="",verbatim=None,*,ARG=None,AT=None,KAPPA=None,NUMERICAL_DERIVATIVES=False,SLOPE=None,STRIDE=None,**kwargs):
  ret=""
  ret+=_format_label(self,LABEL)
  ret+="RESTRAINT"
  retlist=[]
  retlist.append(_format_numbered(self,"ARG",ARG))
  retlist.append(_format_opt(self,"AT",AT))
  retlist.append(_format_opt(self,"KAPPA",KAPPA))
  retlist.append(_format_flag(self,"NUMERICAL_DERIVATIVES",NUMERICAL_DERIVATIVES))
  retlist.append(_format_opt(self,"SLOPE",SLOPE))
  retlist.append(_format_opt(self,"STRIDE",STRIDE))
  for arg in kwargs:
      import re
      allowed=[]
      allowed.append("ARG")
      allowed.append("AT")
      allowed.append("KAPPA")
      allowed.append("NUMERICAL_DERIVATIVES")
      allowed.append("SLOPE")
      allowed.append("STRIDE")
      if not re.sub("[0-9]*$","",arg) in allowed:
         raise TypeError("unknown arg " + arg)
      retlist.append(_format_anything(self,arg,kwargs[arg]))
  retlist.sort()
  for x in retlist:
      if(len(x)>0):
 

These functions are then bound to be methods of the `InputBuilder` class and can be used as we have seen in the examples above.

## A note on efficiency

Efficiency in creating the input file should not be a problem. However, notice that creating an object of class `InputBuilder()` takes a fraction of a second in order to setup all the action methods.

In [45]:
import timeit
print("Time to create an InputBuilder: " + str(timeit.timeit(lambda: plumed.InputBuilder(),number=10)/10))

Time to create an InputBuilder: 0.13217849190000094


If you want to make a script that creates a lot of different input files and then use them on a small number of frames this might be relevant. In this case, you might want to create a lighter version of the builder that does not encode any dictionary using the `load_dict=False` option.

In [46]:
import timeit
print("Time to create an InputBuilder: " + str(timeit.timeit(lambda: plumed.InputBuilder(load_dict=False),number=10)/10))

Time to create an InputBuilder: 0.0011736058999986198


This builder has some limitations in the fact that it does not allow autocompletion and does not contain the documentation of the PLUMED actions. However, it can be used identically to a regular builder

In [47]:
fast=plumed.InputBuilder(load_dict=False)

In [48]:
fast.TORSION("phi",ATOMS="5,7,9,15")

'phi: TORSION ATOMS=5,7,9,15\n'

In [49]:
fast.MOVINGRESTRAINT(ARG="d1",KAPPA=ib.numbered([0,10.0]),AT=ib.numbered([20,20]),STEP=ib.numbered([1,10000]))

'MOVINGRESTRAINT ARG=d1 AT0=20 AT1=20 KAPPA0=0 KAPPA1=10.0 STEP0=1 STEP1=10000\n'